## Torchtext and spacy for preprocessing for NLP
1. `spacy` is the wonderful processing library for NLP
2. `Torchtext` is the wonderful dataset creatation tools for `pytorch` or other frame (`tensorflow`, ...)
3. Functionalities for `Torchtext`
    * File Loading: Load the corpus from various framous formats
    * Tokenization: `spacy` can be more powerful
    * Vocab: generator a vocabulary list
    * Numericalize / Indexify: Map words into integer numbers of index in the entire corpus
    * Word Vector: randomly initialize or pre-train 
    * Batching: padding is normally happening here
4. refer
    * [Torchtext document](https://torchtext.readthedocs.io/en/latest/data.html#fields)
    * [Torchtext chinese blog](https://somewayqxq.com/2017/torchtext-doc/)
    * [Torchtext foreign blog](http://mlexplained.com/2018/02/08/a-comprehensive-tutorial-to-torchtext/)
    * [Allen NLP Torchtext](http://anie.me/On-Torchtext/)
    * [My repo](https://github.com/gmftbyGMFTBY/MiniNMT)
    * [GitHub practice for torchtext](https://github.com/keitakurita/practical-torchtext)
    * [GitHub Tutorial for Torchtext](https://github.com/mjc92/TorchTextTutorial)
    
![](./torchtext.jpg)

In [117]:
# import spacy and load the english
# We only use spacy's tokenizer function here
import spacy
spacy_en = spacy.load("en")

from torchtext import data

### Outline for this tutorial 
* **Field**
* **Dataset**
* **Example**
* **Iterator**
* **Examples**

#### 1. **Field**
```python
data.Field(sequential=True, use_vocab=True, init_token=None, 
           eos_token=None, fix_length=None, dtype=torch.int64,
           preprocessing=None, postprocessing=None, 
           lower=False, tokenize=fucntion instance, 
           include_lengths=False, batch_forst=False, 
           pad_token='<pad>', unk_token='<unk>', 
           pad_first=False, stop_words=None, ...)
```

* `sequential`:  
这个域的数据是否是序列，如果是序列使用 `tokenize` 函数进行分割，否则视为是数字等其他的数据类型，默认是 `True`
* `use_vocab`:  
这个域 (`Field`) 的数据是否是一个词典对象，`False` 说明数据已经是数字类型，默认是 `True`
* `init_token, eos_token, pad_token, unk_token`:  
特殊 `token`, 其中 `init_token, eos_token` 需要指定，`pad_token, unk_token` 可以使用默认值
* `fix_length`:  
`padding` 指定的特定长度
* `dtype`:  
指定数据类型，默认是 `torch.long`
* `preprocessing`:  
这个预处理管道函数用来在 `tokenizer` 之后 `numericalizing` 之前执行，用于自定义的预处理
* `postprocessing`:  
在 `numericaling` 之后 `number` 转换成 `tensor` 之前执行，输入是 `batch`
* `lower`:  
是否将输入的 `text` 全部小写化，默认是 `False`
* `tokenize`:  
用来 `tokenize` 输入的文本，默认是 `str.split` 一般替换成 `spacy`
* `include_lengths`:  
`True` 返回的数据是 `(padded minibatch, lengths of each examples)`, `False` 返回的是 `padded minibatch`
* `batch_first`:  
`False` 默认，`shape` 是 `[, batch]`, `True shape, [batch, ]`
* `build_vocab(train, vectors, min_freq, max_size)`:  
该方法是用来在该 `Field` 上生成对应的词典，需要使用 `train / test/ valid` 数据集对象，这个我们之后会提到，在这个数据集对象上建立对应的词典, `vectors` 指定预训练的词向量 (`glove.6B.100d`).`min_freq` 规定了词表中的最小的单词频率，过滤掉了出现频次小的单词,`max_size` 规定了词表的最大大小，如果超过了的话，按照出现的频次选取 `top-k` 个，频率最前面的单词会出现在词表中。
* `vocab`:  
这是一个字典，这个字典在 `build_vocab` 函数调用之后会得到，这个字典最有名的方法就是
    * `length`: 
    ```python
    len(Field.vocab)
    ```
    * `itos`:
    ```python
    # index to string
    Field.itos[0]
    ```
    * `stoi`:
    ```python
    Field.stoi['string']
    ```
    * `vectors`:  
    当 `build_vaocb` 的时候使用了预训练的词向量的时候这一项就是词向量的 `lookup table` 之后可以赋值给 `torch.Embedding`
    * `freqs`:  
    对 `vocab` 中的数据统计出现的次数

In [118]:
def tokenizer(text):
    # use spacy tokenizer function
    return [tok.text for tok in spacy_en.tokenizer(text)]

# This case is from `text-classification` file
TEXT = data.Field(sequential=True, tokenize=tokenizer, lower=True)
LABEL = data.Field(sequential=False, use_vocab=False)

TEXT.pad_token, TEXT.unk_token, TEXT.dtype, TEXT.use_vocab

('<pad>', '<unk>', torch.int64, True)

####  2. **Dataset**
`Dataset` 对象是 `torchtext` 中非常重要的一个对象，主要分为自定义 `Dataset` 和 `TabularDataset`
1. `TabularDataset`  
从 `CSV, TSV, JSON` 数据存储格式中加载对应使用 `columns` 存储的数据
```python
data.TabularDataset(path, format, fields, skip_header=False, ...)
```
    * `path`:  
数据文件的路径
    * `format(str)`:  
数据文件的存储格式 `csv, tsv, json`
    * `fields`:  
    ```python
    [('field_name', field_object), ...]
    ```
    使用的是 `list` 只能是 `csv, tsv` 格式的文件，并且和文件内部的列对应
    ```python
    dict[str: tuple(str, Field)]
    ```
    如果是字典类型，`key` 是 `JSON / CSV / TSV` 中的键名或者列名
    * `skip_header(bool)`:  
    是否跳过首行的文本，比如 `CSV, TSV` 文件中的列说明
    * `splits(path, train, validation, test, format, fields)`:  
    根据下面的具体实例可以看出，该方法用来对 `test, validation, train` 数据集进行对应的分割生成对应的**数据集对象**
    
2. `Dataset`  
使用 `Examples` 对象构成的列表创建 `dataset` 对象，自定义程度更高
```python
data.Dataset(examples, fields, filter_pred=None)
```
    * `examples`:   
    `Example` 对象的列表，之后会提到
    * `fields`:  
    ```python
    list(tuple(str, Field))
    ```
    * `filter_pred(function or None)`:  
    当 `Example` 对象经过过滤函数 `filter_pred` 函数是 `True` 的时候允许加入数据集 

In [119]:
# load the dataset with TabularDataset
tv_datafields = [("TEXT", TEXT), ("LABEL", LABEL)]
train = data.TabularDataset(path='data/train.csv', format='csv', skip_header=True, fields=tv_datafields)

TEXT.build_vocab(train)
TEXT.vocab.freqs

Counter({'what': 2,
         'the': 1,
         'fuck': 2,
         'are': 2,
         'you': 10,
         'doing': 1,
         'here': 2,
         '!': 3,
         'really': 3,
         'piss': 1,
         'me': 1,
         'off': 1,
         'thank': 2,
         'a': 1,
         'good': 1,
         'guy': 2,
         'shut': 1,
         'up': 1,
         'i': 1,
         'am': 1,
         'so': 1,
         'sorry': 1,
         'to': 2,
         'hear': 1,
         'about': 1,
         'that': 1,
         'bye': 1,
         'see': 1,
         'tommerow': 1,
         'get': 1,
         'out': 1,
         'of': 1,
         'stupid': 1,
         'very': 1,
         'much': 1,
         'for': 2,
         'do': 1,
         'us': 1,
         '?': 1,
         'need': 1,
         'youself': 1,
         'no': 1,
         'god': 1})

In [120]:
# load the dataset with Dataset
# extract the examples for the Dataset
examples = []
with open("data/train.csv") as f:
    for line in f.readlines():
        text, label = line.split(',')
        try:
            label = int(label)
        except:
            continue
        examples.append(data.Example.fromlist([text, label], fields=[("TEXT", TEXT), ("LABEL", LABEL)]))

train = data.Dataset(examples, fields=[("TEXT", TEXT), ("LABEL", LABEL)])

# build the vocab on this field
TEXT.build_vocab(train)

# build the vocab for the Field using the datasets
# vectors need to download, which we use the randomly init
TEXT.build_vocab(train)
print(len(TEXT.vocab))
print(TEXT.vocab.stoi["<unk>"])
print(TEXT.vocab.itos[1])
TEXT.vocab.freqs

46
0
<pad>


Counter({'"': 22,
         'what': 2,
         'the': 1,
         'fuck': 2,
         'are': 2,
         'you': 10,
         'doing': 1,
         'here': 2,
         '!': 3,
         'really': 3,
         'piss': 1,
         'me': 1,
         'off': 1,
         'thank': 2,
         'a': 1,
         'good': 1,
         'guy': 2,
         'shut': 1,
         'up': 1,
         'i': 1,
         'am': 1,
         'so': 1,
         'sorry': 1,
         'to': 2,
         'hear': 1,
         'about': 1,
         'that': 1,
         'bye': 1,
         'see': 1,
         'tommerow': 1,
         'get': 1,
         'out': 1,
         'of': 1,
         'stupid': 1,
         'very': 1,
         'much': 1,
         'for': 2,
         'do': 1,
         'us': 1,
         '?': 1,
         'need': 1,
         'youself': 1,
         'no': 1,
         'god': 1})

#### 3. **Example**

我们可以手动的构建每一个数据集的 `Example` 之后合并成数据集，正如上面 `Dataset` 所示
* `Example.fromlist(data, fields)`:  
从 `list` 按照 `Field` 构建对应的 `example`
* `Example.fromdict`:  
从 `dict` 按照 `Field` 构建对应的 `example`
* `fromCSV, fromJSON, fromtree, ...`

In [121]:
string = ["I can tell you a story", 1]
example1 = data.Example.fromlist(string, fields=[("TEXT", TEXT), ("LABEL", LABEL)])

string = {"TEXT": "I can tell you a story", "LABEL": 1}
example2 = data.Example.fromdict(string, fields={'TEXT': ("TEXT", TEXT), "LABEL": ("LABEL", LABEL)})

print(example1.TEXT, example1.LABEL)
print(example2.TEXT, example2.LABEL)

['i', 'can', 'tell', 'you', 'a', 'story'] 1
['i', 'can', 'tell', 'you', 'a', 'story'] 1


#### 4. **Iterator**
构建迭代器
* `datasets`:  
`tuple` 类型，表示 `train, test, validation` 三种不同的数据集类型，下面实例只是用了 `train`
* `sort_key`:  
排序的方法，这里一般都是使用 `length` 作为文本的排序手段以便之后的高效 `padding`
* `batch_sizes`:  
一样，这里是对不同的数据集使用不同的 `batch size` 下面的实例是对 `train` 数据集使用 32 大小的 `batch`
* `train`:  
是否每一个 `iterator` 表示一个训练集
* `repeat`:  
是否允许在 `multiple epochs` 的时候重复使用一个 `batch`
* `shuffle`:  
是否打乱 `examples`
* `device`:  
使用的设备是什么，默认是在 `cpu` 上，`-1` 是 `cpu`
* `sort`:  
是否按照 `sort_key` 进行排序

![](./torchtext.jpg)

最常见的 `Iterator` 就是 `BucketIterator` 因为，在做 `batch` 的 `padding` 的时候我们都知道，如果 `padding token` 越多效率越低，在这里使用 `BucketIterator` 的话在内部会将长度一样的一些 `sentence` 尽量的放入一个 `batch` 中，这样的话可以最小化 `padding token` 的数目，提高训练的效率，参数和 `Iterator` 的参数是一样的，所有的操作都是内部完成的。**如果想要启动这一个特性的话，`sort` 参数必须是 `True`**。但是本质上 `BucketIterator, Iterator`两者没有什么差别，实验效果一样，怀疑是对 `Iterator` 也做了相应的优化。

In [138]:
def test(train_iter):
    # print the examples in train iterations
    # 按照上面的图示，每一个元素都是一个 batch，但是因为我们，每一个 batch 使用最长的元素进行 padding
    padding_count = 0
    padding_token = TEXT.vocab.stoi["<pad>"]
    
    for batch in train_iter:
        print(batch.LABEL, '\n', batch.TEXT)
        # show one result from the examples
        example0 = batch.TEXT[:, 0]
        print([TEXT.vocab.itos[item] for item in example0])
        padding_count += (batch.TEXT == padding_token).sum()
    print("Sum of padding token:", padding_count)
    print()
        
train_iter = data.Iterator(train, sort_key=lambda x: len(x.TEXT), batch_size=4, device=-1)
print("Examples of Iterator: ")
test(train_iter)
train_iter = data.BucketIterator(train, sort_key=lambda x: len(x.TEXT), batch_size=4, sort=True)
print("Examples of BucketIterator: ")
test(train_iter)

The `device` argument should be set by using `torch.device` or passing a string as an argument. This behavior will be deprecated soon and currently defaults to cpu.


Examples of Iterator: 
tensor([1, 0, 1, 1]) 
 tensor([[ 2,  2,  2,  2],
        [13, 29, 21,  5],
        [40,  4, 32, 14],
        [ 8,  2, 30,  3],
        [ 6,  1, 10,  5],
        [ 3,  1,  3, 28],
        [20,  1, 38, 12],
        [10,  1,  9,  8],
        [ 4,  1,  2, 45],
        [ 2,  1,  1,  2]])
['"', 'what', 'the', 'fuck', 'are', 'you', 'doing', 'here', '!', '"']
tensor([1, 0, 0, 0]) 
 tensor([[ 2,  2,  2,  2],
        [ 3, 18, 25, 11],
        [ 5, 34, 17,  3],
        [33,  3, 36,  3],
        [26, 41, 37,  6],
        [31,  2, 12, 15],
        [ 2,  1, 24, 23],
        [ 1,  1, 16,  9],
        [ 1,  1, 39,  2],
        [ 1,  1,  2,  1]])
['"', 'you', 'really', 'piss', 'me', 'off', '"', '<pad>', '<pad>', '<pad>']
tensor([0, 0, 1]) 
 tensor([[ 2,  2,  2],
        [22, 11,  3],
        [ 4,  3, 35],
        [ 2, 44, 42],
        [ 1, 27,  2],
        [ 1,  7,  1],
        [ 1, 13,  1],
        [ 1,  3,  1],
        [ 1, 19,  1],
        [ 1,  7,  1],
        [ 1, 43,  1],
 

#### 5. **Examples**
show some examples